# Zero Shot examples with HelloWorld-MDB
## Overview
We will use 1 sample application for this example.  It has an original branch which represents the starting point as a Java EE application and then another branch for the end result of being migrated to Quarkus.  
## What are we doing?
* We will experiment with several zero-shot prompts to see what kind of information we can get back from the model.
    * We will run this example with 2 models from OpenAI and save the output to disk
* What are the prompts we will use:
    1. Simple prompt with little added to it beyond the java ee source code and a simple directive
    2. A slightly improved prompt in that we direct the LLM to think about the reasoning prior to updating the code
    3. An improved prompt that combines issues we identified via our static code analysis
## Sample App Information
### Hello World MDB
* This example started from JBoss EAP Quickstarts: https://github.com/jboss-developer/jboss-eap-quickstarts/tree/7.4.x/helloworld-mdb/src/main/java/org/jboss/as/quickstarts/mdb
* Our team then ported 'HelloWorld MDB' to Quarkus:https://github.com/savitharaghunathan/helloworld-mdb/tree/quarkus
    * Note these blog posts from prior work of modernizing this same HelloWorld app, the difference with our latest work and the below is the change our team did to also address JMS to Reactive changes:
        * https://developers.redhat.com/blog/2019/11/07/quarkus-modernize-helloworld-jboss-eap-quickstart-part-1#let_s_modernize_helloworld
        * https://developers.redhat.com/blog/2019/11/08/quarkus-modernize-helloworld-jboss-eap-quickstart-part-2
* We are running some custom rules with this analysis:
    * The analysis run via the script [../data/custom_rule_analyze_helloworld.sh](../data/custom_rule_analyze_helloworld.sh)
    * We developed rules for JMS -> Reactive here:
        * https://github.com/windup/windup-rulesets/pull/1043
        * https://github.com/jmle/rulesets/blob/jms-rule/default/generated/quarkus/05-jms-to-reactive-quarkus.windup.yaml



In [1]:
# Bring in the common deps
import os
import pprint

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

from kyma_poc import Report
from kyma_poc.scm import GitDiff

pp = pprint.PrettyPrinter(indent=2)

def get_java_in_result(text: str):
    _, after = text.split("```java")
    return after.split("```")[0]

def write_output_to_disk(out_dir, formatted_template, example_javaee_file_contents, example_quarkus_file_contents, result):
    try:
        os.makedirs(out_dir, exist_ok=True)
    except OSError as error:
        print(f"Error creating directory {out_dir}: {error}")
        raise error

    # Save the template to disk
    template_path = os.path.join(out_dir, "template.txt")
    with open(template_path, "w") as f:
        f.truncate()
        f.write(formatted_template)
    print(f"Saved template to {template_path}")

    # Save the result
    result_path = os.path.join(out_dir, "result.txt")
    with open(result_path, "w") as f:
        f.truncate()
        f.write(result)
    print(f"Saved result to {result_path}")

    # Extract the updated java code and save it
    updated_file_contents = get_java_in_result(result)
    updated_file_path = os.path.join(out_dir, "updated_file.java")
    with open(updated_file_path, "w") as f:
        f.truncate()
        f.write(updated_file_contents)
    print(f"Saved updated java file to {updated_file_path}")

    # Save the original source code
    original_file_path = os.path.join(out_dir, "original_file.java")
    with open(original_file_path, "w") as f:
        f.truncate()
        f.write(example_javaee_file_contents)
    print(f"Saved original java file to {original_file_path}")

    # Save the Quarkus version we already in Git to use as a comparison
    known_quarkus_file_path = os.path.join(out_dir, "quarkus_version_from_git.java")
    with open(known_quarkus_file_path, "w") as f:
        f.truncate()
        f.write(example_quarkus_file_contents)
    print(f"Saved Quarkus version from Git to {known_quarkus_file_path}")


model_name="gpt-4-1106-preview"
#model_name="gpt-3.5-turbo-16k"

example_git_path = "../data/helloworld-mdb"
gd = GitDiff(example_git_path)

# Pick a file to update
example_file_path = "src/main/java/org/jboss/as/quickstarts/mdb/HelloWorldQueueMDB.java"
# JavaEE code is in 'main' branch,  Quarkus code is in 'quarkus' branch
example_file_javaee_contents = gd.get_file_contents(example_file_path, gd.get_commit_from_branch("main"))
# Below represents the 'solved' state of the file which was manually migrated to Quarkus
# We will compare to this to see how well we did with the generation example
example_file_quarkus_contents = gd.get_file_contents(example_file_path, gd.get_commit_from_branch("quarkus"))

# TODO:
# Add a step to run TreeSitter on the updated java file to see if it parses OK


# Zero Shots
We plan to run 3 Zero Shot examples:
1. Simplest prompt we can imagine with basic instructions
2. Improved prompt that has a little more context and instructions
3. Add in hints from Static Code Analysis of what to change

## 1. Zero Shot - Very Simple Prompt
* Let's start with a simple prompt that has very little in it

In [2]:
output_dir = f"output/{model_name}/helloworldmdb/zero_shot_naive/very_simple/"

very_simple_template = """
    Migrate the passed in enterprise Java EE source code to Quarkus

    ## Original input file in Java EE to update to Quarkus
    Filename: "{file_path}"

    ### Source code
    ```java 
    {example_file_contents}
    ```

    Structure your ouput in Markdown format such as:
    ## Updated File
    ```java
        Write the updated file for Quarkus in this section
    ```
   
    """

template_args = {
    "file_path": example_file_path,
    "example_file_contents": example_file_javaee_contents
}
formatted_prompt = very_simple_template.format(**template_args)

llm = ChatOpenAI(temperature=0.1, model_name=model_name)
prompt = PromptTemplate.from_template(very_simple_template)
chain = LLMChain(llm=llm, prompt=prompt)
result = chain.run(template_args)

write_output_to_disk(output_dir, formatted_prompt, example_file_javaee_contents, example_file_quarkus_contents, result)


Saved template to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/very_simple/template.txt
Saved result to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/very_simple/result.txt
Saved updated java file to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/very_simple/updated_file.java
Saved original java file to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/very_simple/original_file.java
Saved Quarkus version from Git to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/very_simple/quarkus_version_from_git.java


## 2. Zero Shot - Improved prompt, LLM asked to share it's reasoning
* Prompt is still relatively simple but we ask the LLM to think through the reasoning first before responding

In [3]:
output_dir = f"output/{model_name}/helloworldmdb/zero_shot_naive/improved/"

improved_template = """
    You are an AI Assistant trained on migrating enterprise JavaEE code to Quarkus.
    I will give you an example of a JavaEE file and you will give me the Quarkus equivalent.
    Approach this code migration from Java EE to Quarkus as if you were an experienced developer.
    Before attempting to migrate the code to Quarkus, explain each step that is required and why.
    After you have shared your step by step thinking, provide a full output of the updated file:

    ## Original input file in Java EE to update to Quarkus
    Filename: "{file_path}"

    ### Source code
    ```java 
    {example_file_contents}
    ```

    Structure your ouput in Markdown format such as:

    ## Reasoning
    Write the step by step reasoning in this markdown section.
    If you are unsure of a step or reasoning, clearly state you are unsure and why.

    ## Updated File
    ```java
        Write the updated file for Quarkus in this section
    ```
   
    """

template_args = {
    "file_path": example_file_path,
    "example_file_contents": example_file_javaee_contents
}
formatted_prompt = improved_template.format(**template_args)

llm = ChatOpenAI(temperature=0.1, model_name=model_name)
prompt = PromptTemplate.from_template(improved_template)
chain = LLMChain(llm=llm, prompt=prompt)
result = chain.run(template_args)

write_output_to_disk(output_dir, formatted_prompt, example_file_javaee_contents, example_file_quarkus_contents, result)

Saved template to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/improved/template.txt
Saved result to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/improved/result.txt
Saved updated java file to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/improved/updated_file.java
Saved original java file to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/improved/original_file.java
Saved Quarkus version from Git to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/improved/quarkus_version_from_git.java


## 3. Zero Shot with Static Code Analysis Info
* We will pick a single file and form a template that includes information from our static code analysis to ask the LLM to help update towards Quarkus


In [4]:
output_dir = f"output/{model_name}/helloworldmdb/zero_shot_naive/improved_with_analysis/"

## Get the issues found for a given file in the sample app
## Assumes we have already run kantra via `data/custom_rule_analyzer_helloworld.sh`
hello_world_mdb_jms_yaml = "../data/example_reports/helloworld-mdb_custom_rules/output.yaml"
reportObj = Report(hello_world_mdb_jms_yaml)
impacted_files = reportObj.get_impacted_files()
# Our example file should be in the impacted files, let's be sure or fail now
assert example_file_path in impacted_files.keys()
issues = impacted_files[example_file_path]

counter = 0
issues_text = "We have identified the following issues which should be considered when migrating the source code to Quarkus\n"
for issue in issues:
    counter += 1
    issues_text += f"Issue #{counter}:\n"
    issues_text += f"    {issue['message']}\n"
    issues_text += f"    {issue['violation_description']}\n"

template_with_analysis = """
    # Java EE to Quarkus Migration
    You are an AI Assistant trained on migrating enterprise JavaEE code to Quarkus.
    I will give you an example of a JavaEE file and you will give me the Quarkus equivalent.
    Pay attention to changes you make and impacts to external dependencies in the pom.xml.  
    As you make changes that impact the pom.xml be sure you explain what needs to be updated in pom.xml.
    
    In addition, I will provide you with a list of issues that were found in the JavaEE code which
    we believe need to be addressed before being migrated to Quarkus.  Be sure to address these 
    issues or explain why you do not think the issue is relevant.

    Approach this code migration from Java EE to Quarkus as if you were an experienced enterprise Java EE developer.
    Before attempting to migrate the code to Quarkus, explain each step that is required and why.
    After you have shared your step by step thinking, provide a full output of the updated file:

    ## Original input file in Java EE to update to Quarkus
    Filename: "{file_path}"

    ### Source code
    ```java 
    {example_file_contents}
    ```

    ## Issues found from static code analysis of the Java EE code
    {issues_text}

    # Output Instructions
    Structure your ouput in Markdown format such as:

    ## Reasoning
    Write the step by step reasoning in this markdown section.
    If you are unsure of a step or reasoning, clearly state you are unsure and why.

    ## Updated File
    ```java
        Write the updated file for Quarkus in this section
    ```
   
    """

template_args = {
    "file_path": example_file_path,
    "example_file_contents": example_file_javaee_contents,
    "issues_text": issues_text
}
formatted_prompt = template_with_analysis.format(**template_args)

llm = ChatOpenAI(temperature=0.1, model_name=model_name)
prompt = PromptTemplate.from_template(template_with_analysis)
chain = LLMChain(llm=llm, prompt=prompt)
result = chain.run(template_args)

write_output_to_disk(output_dir, formatted_prompt, example_file_javaee_contents, example_file_quarkus_contents, result)

Reading report from ../data/example_reports/helloworld-mdb_custom_rules/output.yaml
Saved template to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/improved_with_analysis/template.txt
Saved result to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/improved_with_analysis/result.txt
Saved updated java file to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/improved_with_analysis/updated_file.java
Saved original java file to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/improved_with_analysis/original_file.java
Saved Quarkus version from Git to output/gpt-3.5-turbo-16k/helloworldmdb/zero_shot_naive/improved_with_analysis/quarkus_version_from_git.java
